# Deploy model as a webservice on Azure Kubernetes Service

## Table of contents
1. [Prerequisites](#prerequisites)

2. [Initialize workspace](#workspace)

3. [Deploy Model to AKS](#deploymodel)

- a) [Create scoring file](#scoringfile)
- b) [Define Enviroment](#env)
- c) [Deployment configuration](#configfile)
- d  [Deploy Webservice](#webservice)
- e) [Test Webservice](#testservice)



### 1. Prerequisites <a id='prerequisites'></a>

In [2]:
import numpy as np
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.41.0


### 2. Initialize workspace <a id='workspace'></a>

In [3]:
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep = '\n')

ml-ws-aiplatform-dev
rg-aiplatform-dev
northeurope


### 3. Deploy model <a id='deploymodel'></a>

#### a) Create a scoring script <a id='scoringfile'></a>

In [4]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib
import onnxruntime
import time
from azureml.core.model import Model
from azureml.monitoring import ModelDataCollector
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model, scaler, input_name, label_name, inputs_dc, prediction_dc
    

    scaler_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model-scaler/1/model-scaler.pkl')
    # deserialize the model file back into a sklearn model
    scaler = joblib.load(scaler_path)
    
    model_onnx = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'support-vector-classifier/1/svc.onnx')
    # print(os.listdir(model_onnx))
    model = onnxruntime.InferenceSession(model_onnx, None)
    input_name = model.get_inputs()[0].name
    label_name = model.get_outputs()[0].name
    
    # variables to monitor model input and output data
    inputs_dc = ModelDataCollector("Support vector classifier model", designation="inputs", feature_names=["feat1", "feat2", "feat3", "feat4", "feat5", "feat6", "feat7"])
    prediction_dc = ModelDataCollector("Support vector classifier model", designation="predictions", feature_names=["weatherprediction"])

    
@input_schema('data', NumpyParameterType(np.array([[34.927778, 0.24, 7.3899, 83, 16.1000, 1016.51, 1]])))
@output_schema(NumpyParameterType(np.array([0])))
def run(data):
                try: 
                    data = scaler.fit_transform(data.reshape(1, 7))
                    inputs_dc.collect(data)
                    
                    # model inference
                    result = model.run([label_name], {input_name: data.astype(np.float32)})[0]
                    # this call is saving model output data into Azure Blob
                    prediction_dc.collect(result)

                 
                except Exception as e:   
                    result = 'error'
                    prediction_dc.collect(result)
                    
                return result.tolist()            


Overwriting score.py


#### b) Define Environment <a id='env'></a>

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

Environment(name="myenv")

#env = Environment.get(workspace=ws, name="AzureML-Minimal")
env = Environment.get(workspace=ws, name="AzureML-Minimal").clone('myenv')



In [ ]:
for pip_package in ["numpy", "onnxruntime", "joblib", "azureml-core", "azureml-monitoring", "azureml-defaults", "scikit-learn==0.20.3", "inference-schema", "inference-schema[numpy-support]"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

#### c) Deployment Configuration <a id='configfile'></a>

In [ ]:
from azureml.core.webservice import AksWebservice

aks_config = AksWebservice.deploy_configuration(collect_model_data=True)

#### d) Deploy web service <a id='webservice'></a>

In [10]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AksCompute, ComputeTarget

# Choose a name for your AKS cluster
aks_name = 'port-aks' 

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


In [12]:
model1 = Model(ws, 'scaler')
model2 = Model(ws, 'support-vector-classifier')

service_name = 'weather-aks-prediction'

In [ ]:
service = Model.deploy(ws, service_name, models=[model1, model2], inference_config=inference_config, deployment_config=aks_config, deployment_target=aks_target,overwrite=True)
service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
print(service.get_logs())

In [ ]:
service.update(enable_app_insights=True)

#### e) Test web service <a id='testservice'></a>

In [ ]:
print(service.scoring_uri)

In [ ]:
print(service.swagger_uri)

In [ ]:
service.state

In [ ]:
import json


input_payload = json.dumps({
    'data': [[34.927778, 0.24, 7.3899, 83, 16.1000, 1016.51, 1]],
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

In [ ]:
# service.delete()